In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageOps # Necesitas: pip install pillow

def predecir_ropa(ruta_imagen):
    # 1. CARGAR EL MODELO
    # Cargamos el cerebro que ya entrenaste
    model = tf.keras.models.load_model('mi_modelo.keras')

    # 2. CARGAR Y PRE-PROCESAR LA IMAGEN (La parte crítica)
    # A. Abrir imagen como escala de grises ('L')
    img = Image.open(ruta_imagen).convert('L')

    